In [ ]:
#Importação das bibliotecas necessárias
import pandas as pd                      # Manipulação de dados (DataFrame)
import numpy as np                       # Cálculos numéricos
from sklearn.impute import SimpleImputer # Tratamento de valores faltantes
from sklearn.preprocessing import StandardScaler, OrdinalEncoder # Normalização e codificação
import joblib                            # Salvamento do scaler
import seaborn as sns                    # Visualização (opcional)
import matplotlib.pyplot as plt          # Gráficos (opcional)
import os


In [ ]:
#Carregamento do dataset

# Lemos o arquivo CSV original com os dados de e-commerce.
df = pd.read_csv("ecommerce_sales.csv")

# Exibimos as 5 primeiras linhas para conferir a estrutura inicial do dataset.
print("Visualização inicial do dataset:")
print(df.head(), "\n")

# Informações gerais sobre colunas, tipos de dados e valores nulos.
df.info()

# Estatísticas descritivas básicas (inclui colunas numéricas e categóricas)
print("\nResumo estatístico:")
print(df.describe(include='all').T)

Visualização inicial do dataset:
     sale_id  marketing_spend  website_traffic  conversion_rate  \
0  SALE00022          5736.77          15473.0             5.05   
1  SALE00633          6553.71          16501.0             7.33   
2  SALE01551              NaN          12624.0             5.82   
3  SALE00391          7141.07          21362.0             4.82   
4  SALE00375          1232.50          17901.0             4.01   

   avg_product_rating  discount_percentage  num_products  avg_price  \
0                2.34                   25            90     129.97   
1                3.58                   10            37     118.24   
2                2.56                   15           499      97.37   
3                2.90                   20           128      64.31   
4                3.66                    0           429     161.47   

   customer_reviews  return_rate product_category competition_level  \
0             795.0        10.48          Fashion            Mediu

In [ ]:
 #Tratamento de Valores Faltantes

print("\nEtapa 1 - Tratamento de Valores Faltantes")

# Identifica colunas numéricas e categóricas
num_cols = df.select_dtypes(include=['int64', 'float64']).columns
cat_cols = df.select_dtypes(include=['object']).columns

# Cria um imputador para colunas numéricas com estratégia de mediana
num_imputer = SimpleImputer(strategy='median')
df[num_cols] = num_imputer.fit_transform(df[num_cols])

# Cria um imputador para colunas categóricas com estratégia de moda (valor mais frequente)
cat_imputer = SimpleImputer(strategy='most_frequent')
df[cat_cols] = cat_imputer.fit_transform(df[cat_cols])

# Verifica novamente se ainda existem valores nulos
print("\nValores faltantes após imputação:")
print(df.isna().sum().sort_values(ascending=False))



Etapa 1 - Tratamento de Valores Faltantes

Valores faltantes após imputação:
sale_id                0
marketing_spend        0
website_traffic        0
conversion_rate        0
avg_product_rating     0
discount_percentage    0
num_products           0
avg_price              0
customer_reviews       0
return_rate            0
product_category       0
competition_level      0
seasonality            0
free_shipping          0
payment_methods        0
mobile_traffic_pct     0
monthly_sales          0
dtype: int64


In [5]:
# Tratamento de Outliers
#---------------------------------------------------------------
print("\nEtapa 2 - Tratamento de Outliers")

# Define uma função para aplicar o método de "capping" (limitação dos outliers)
def tratar_outliers(df, col):
    """
    Essa função identifica os outliers com base no intervalo interquartil (IQR)
    e limita os valores da coluna entre os limites inferiores e superiores.
    """
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    limite_inf = Q1 - 1.5 * IQR
    limite_sup = Q3 + 1.5 * IQR
    
    # Aplica o capping
    df[col] = df[col].clip(limite_inf, limite_sup)
    return df

# Aplica a função para as principais colunas numéricas do dataset
for col in ['marketing_spend', 'website_traffic', 'avg_price', 'monthly_sales']:
    if col in df.columns:
        df = tratar_outliers(df, col)

print("Outliers tratados com sucesso nas principais colunas numéricas!")


Etapa 2 - Tratamento de Outliers
Outliers tratados com sucesso nas principais colunas numéricas!


In [ ]:
# Encoding de Variáveis Categóricas
# ---------------------------------------------------------------
print("\nEtapa 3 - Encoding de Variáveis Categóricas")

# Dicionário de variáveis ordinais, caso existam (exemplo)
ordinais = {
    # 'satisfacao_cliente': ['ruim', 'médio', 'bom', 'excelente']
}

# Aplica Ordinal Encoding nas colunas definidas como ordinais
for col, ordem in ordinais.items():
    encoder = OrdinalEncoder(categories=[ordem])
    df[[col]] = encoder.fit_transform(df[[col]])

# Aplica One-Hot Encoding nas demais variáveis categóricas
# drop_first=True evita multicolinearidade removendo a primeira categoria
df = pd.get_dummies(df, columns=[col for col in cat_cols if col not in ordinais], drop_first=True)

print("Variáveis categóricas transformadas com sucesso!")
print("Dimensões do dataset após encoding:", df.shape)

# ---------------------------------------------------------------


Etapa 3 - Encoding de Variáveis Categóricas
Variáveis categóricas transformadas com sucesso!
Dimensões do dataset após encoding: (2510, 2568)


In [ ]:
# Cria a pasta 'models' se ela não existir
os.makedirs('models', exist_ok=True)

# Agora salva o scaler
joblib.dump(scaler, 'models/scaler.pkl')

['models/scaler.pkl']

In [ ]:

# Cria a pasta se não existir
os.makedirs("data/processed", exist_ok=True)

# Salva o dataframe atual (df) como dados_tratados.csv
df.to_csv("data/processed/dados_tratados.csv", index=False)

print(" Arquivo 'dados_tratados.csv' criado com sucesso em 'data/processed/'.")

 Arquivo 'dados_tratados.csv' criado com sucesso em 'data/processed/'.


In [13]:
# ---------------------------------------------------------------
# Etapa 4 - Normalização de Variáveis Numéricas
# ---------------------------------------------------------------
# Objetivo:
# Padronizar as colunas numéricas para que fiquem na mesma escala.
# Isso evita que variáveis com grandes valores dominem o modelo.
# ---------------------------------------------------------------

import os
import pandas as pd
from sklearn.preprocessing import StandardScaler
import joblib

# Cria pastas necessárias (caso ainda não existam)
os.makedirs('data/processed', exist_ok=True)
os.makedirs('models', exist_ok=True)
os.makedirs('data/normalized', exist_ok=True)

# Carrega o dataset tratado (resultado das etapas anteriores)
# Se não existir, altere o caminho conforme seu arquivo disponível
df = pd.read_csv('data/processed/dados_tratados.csv')

# Identifica as colunas numéricas
num_cols = df.select_dtypes(include=['float64', 'int64']).columns

# Inicializa o StandardScaler (normalizador)
scaler = StandardScaler()

# Ajusta e transforma os dados
df[num_cols] = scaler.fit_transform(df[num_cols])

# Salva o scaler para uso futuro (em teste ou produção)
joblib.dump(scaler, 'models/scaler.pkl')

# Salva o dataset normalizado
df.to_csv('data/normalized/dados_normalizados.csv', index=False)

print(" Normalização concluída com sucesso!")
print(" Scaler salvo em: models/scaler.pkl")
print("Dados normalizados salvos em: data/normalized/dados_normalizados.csv")

 Normalização concluída com sucesso!
 Scaler salvo em: models/scaler.pkl
Dados normalizados salvos em: data/normalized/dados_normalizados.csv


In [ ]:
# ---------------------------------------------------------------
# Etapa 5 - Encoding de Variáveis Categóricas
# ---------------------------------------------------------------
# Objetivo:
# Converter variáveis categóricas (texto) em valores numéricos.
# Isso é necessário, pois os modelos de ML só trabalham com números.
# ---------------------------------------------------------------


# Garante que a pasta de saída exista
os.makedirs('data/encoded', exist_ok=True)

# Carrega o dataset normalizado
df = pd.read_csv('data/normalized/dados_normalizados.csv')

# Identifica colunas categóricas (tipo object)
cat_cols = df.select_dtypes(include=['object']).columns

# Aplica One-Hot Encoding (para variáveis nominais)
# drop_first=True evita multicolinearidade (exclui uma das categorias)
df_encoded = pd.get_dummies(df, columns=cat_cols, drop_first=True)

# Salva o dataset com encoding aplicado
df_encoded.to_csv('data/encoded/dados_codificados.csv', index=False)

print("Encoding concluído com sucesso!")
print(f"Colunas categóricas convertidas: {list(cat_cols)}")
print(" Dados salvos em: data/encoded/dados_codificados.csv")

Encoding concluído com sucesso!
Colunas categóricas convertidas: []
 Dados salvos em: data/encoded/dados_codificados.csv


In [ ]:
# ---------------------------------------------------------------
# Etapa 6 - Feature Engineering (Criação de Novas Features)
# ---------------------------------------------------------------
# Objetivo:
# Criar novas colunas a partir das existentes que possam melhorar
# a performance do modelo.
# ---------------------------------------------------------------

# Garante que a pasta de saída exista
os.makedirs('data/engineered', exist_ok=True)

# Carrega o dataset codificado
df = pd.read_csv('data/encoded/dados_codificados.csv')

# Exemplo 1: Criar uma feature de eficiência de marketing
if {'marketing_spend', 'monthly_sales'}.issubset(df.columns):
    df['marketing_efficiency'] = df['monthly_sales'] / (df['marketing_spend'] + 1)

# Exemplo 2: Criar uma feature de conversão de tráfego
if {'website_traffic', 'monthly_sales'}.issubset(df.columns):
    df['conversion_rate'] = df['monthly_sales'] / (df['website_traffic'] + 1)

# Salva o dataset com novas features
df.to_csv('data/engineered/dados_engineered.csv', index=False)

print(" Feature Engineering concluída com sucesso!")
print("Novas features criadas: ['marketing_efficiency', 'conversion_rate']")
print("Dados salvos em: data/engineered/dados_engineered.csv")

 Feature Engineering concluída com sucesso!
Novas features criadas: ['marketing_efficiency', 'conversion_rate']
Dados salvos em: data/engineered/dados_engineered.csv
